# GMM, round 2

We've seen the basics of how 2-step GMM works; now we are going to organize our code using the ``class`` and ``module`` structures in python, and walk through another example involving testing model fit. First, organization.

## Organizing code into classes and modules
As we've seen with GMM, there are a lot of "little" functions that help us estimate: ``gj``, ``gN``, ``minimize``... A nice way to make our code more concise and readable is to collect these functions into a python **module**. Modules are just ``.py`` scripts that contain functions, and sometimes even classes. They also import the libraries that these functions depend on. This is what ``gmm.py`` is in Ethan's repository (reproduced in this folder).

Let's take a look...





Now that we have all the building blocks set up, we can systematize calling them by organizing them into a class. First, we need to set up instantiation. Here are some coding tricks that will come in handy:

In [ ]:
# Trick 1: try
# Let's say I have a function that receives an input x,
# but it doesnt know if x is a number or a list.
# This is like our betas; there may be just 1 (a number)
# or L (a list)
# try let's us try to code something, and then tell python 
# what to do if that fails
x = 2
y = [1, 2, 3]

def demonstrateTry(z):
    try:
        print(f"z has len {len(z)}") 
        # try to print the length of x, 
        # which will only work if a list
    except:
        print("z is a scalar")
    # here I do this given any error above, but I can be 
    # more specific about what to do for a given type of error.
    # like: except TypeError:
    
demonstrateTry(x)
demonstrateTry(y)

In [ ]:
try: 
    len(x)
except Exception as e: print(e)

In [ ]:
# Trick 2: Another way to plan for different types of variables
# being passed is to do if statements. Let's say we don't know if
# x is a tuple of data, like (y, X, Z), or just a matrix like X.
import numpy as np

def testType(data):
    if type(data) is tuple:
        X = data[1]
    else:
        X = data
    # print the shape to see that we get back something
    # that makes sense
    print(X.shape)

# make some random data
randomNormal = lambda K: np.random.normal(size = (10, K))    
data1 = (randomNormal(1), randomNormal(2), randomNormal(2)) # tuple
data2 = randomNormal(2) # numpy array

# test
testType(data1)
testType(data2)

First let's look at just the instantiation, that is, the setting of attributes and what gets passed in:

In [ ]:
import gmm
import numpy as np

class GMM: # not necessary to inherit from object as Ethan does
    # Constructor
    def __init__(self,gj,data,B,W=None):
        """GMM problem for restrictions E(gj(b0))=0, 
        estimated using data with b0 in R^k.

           - If supplied B is a positive integer k, then 
             space taken to be R^k.  
           - If supplied B is a k-vector, then
             parameter space taken to be R^k with B a possible
             starting value for optimization.
        """
        # set attribute
        self.gj = gj
        # Overwrite member of gmm module by passing our FUNCTION gj 
        # all the way back to the gmm module
        gmm.gj = gj  
        # set attribute
        self.data = data
        # set attribute
        self.W = W
        # set attribute b to None since we haven't solved for this yet
        self.b = None
        # set attributes based on B
        try:
            self.k = len(B)
            self.b_init = np.array(B)
        except TypeError:
            self.k = B
            self.b_init = np.zeros(self.k)
        # infer the number of moment conditions L (ell)
        # from the no. of columns of g_j
        self.ell = gj(self.b_init,self.data).shape[1]
        # infer the no. of obs from the no. of rows in the data
        # if the data is a tuple (like (y, X, Z)), then we first
        # need to subset the tuple then get the shape
        if type(data) is tuple:
            self.N = data[0].shape[0]
        else:
            self.N = data.shape[0]
        # set the minimizer function 
        self.minimize = gmm.minimize

Now let's add in our methods, which we can conveniently inherit from the GMM module!

In [ ]:
class GMM:

    def __init__(self,gj,data,B,W=None):
        """GMM problem for restrictions E(gj(b0))=0, 
        estimated using data with b0 in R^k.

           - If supplied B is a positive integer k, then 
             space taken to be R^k.  
           - If supplied B is a k-vector, then
             parameter space taken to be R^k with B a possible
             starting value for optimization.
        """
        self.gj = gj
        gmm.gj = gj  # Overwrite member of gmm module
        self.data = data

        self.W = W

        self.b = None

        try:
            self.k = len(B)
            self.b_init = np.array(B)
        except TypeError:
            self.k = B
            self.b_init = np.zeros(self.k)

        self.ell = gj(self.b_init,self.data).shape[1]

        if type(data) is tuple:
            self.N = data[0].shape[0]
        else:
            self.N = data.shape[0]

        self.minimize = gmm.minimize
            
    def gN(self,b):
        """Averages of g_j(b).
        This is generic for data, to be passed to gj.
        """
        return gmm.gN(b,self.data)

    def Omegahat(self,b):
        return gmm.Omegahat(b,self.data)
    
    def J(self,b,W):
        return gmm.J(b,W,self.data)

    def one_step_gmm(self,W=None,b_init=None):
        self.b = gmm.one_step_gmm(self.data,W,b_init=self.b_init)
        return self.b
    
    def two_step_gmm(self):
        self.b = gmm.two_step_gmm(self.data,b_init=self.b_init)[0]
        return self.b

    def continuously_updated_gmm(self):
        self.b = gmm.continuously_updated_gmm(self.data,b_init=self.b_init)[0]
        return self.b

## Another example of a GMM application

Remember: GMM starts with a _hypothesis_; we assert a moment condition that we believe to hold because of logic, a model, etc., and then use data to back out the parameters that best help the data match this assertion.

In IV, our assertion is that the instrument is not correlated with the error. That is, we have reason to believe (because of a model, logic, whatever) that Z only affects Y through X, so we assert this through the moment condition $E(Z'e) = E(Z'(y - X\beta)) = 0$, and solve for the $\beta_k$ that most closely make this assertion hold in the data. Similarly, in our last section, we asserted the first order conditions that "believed" should hold, then solved for the $\alpha$ that best made the data match. 

Here we will propose a new model we want to estimate with GMM, but this time we will make 2 data generating processes: one where the true relationship does hold in the data, and one where it absolutely does not, and see how GMM behaves.

### Model set-up

Consider a non-linear, commonly called Poisson ML process:

$$y_i = exp\{X_i\beta\} + e_i$$

We may posit this relationship primarily if we think that the error enters separately from the exponential for some reason. Alternatively, this is an attractive approximation if we believe there is a relationship between log y and $X_i\beta$, and in particular there are some (or a lot) of zeros in y preventing us from taking logs (environment friends: the inverse hyperbolic sine function is [not always your friend](https://marcfbellemare.com/wordpress/wp-content/uploads/2019/02/BellemareWichmanIHSFebruary2019.pdf)).

Suppose we have an instrument for $X_i$, but we already know from the forbidden regression that we cannot apply IV when the second stage is non-linear! Welcome, GMM.

To be clear, the exogeneity of Z is _our assertion_. That is, we as the researcher have reason to believe that:

$$E(e_i | z_i) = 0 \Rightarrow E(z_i'e_i) = 0$$

So we use this "information" to propose our moment condition:

$$g_j(\beta) = E(z_i'e_i) = E(z_i'(y_i - exp\{X_i\beta\})) = 0$$

In [ ]:
def gj(b, data):
    # make sure b is the right shape
    b = b.reshape(2,1)
    
    y, X, Z = data
    Z_0 = Z[:, [0]]
    Z_1 = Z[:, [1]]
    Z_2 = Z[:, [2]]

    e = y - np.exp(X@b)
    #horizontally stack the instruments for an
    #Nx2 matrix
    return ...

Now we want 2 data-generating processes: one where our instrument is exogenous, and one where exogeneity is violated. To do this, I define a single DGP function, which takes as a parameter the covariance between Z and e, which should be 0 in the exogenous case, and $\neq 0$ in the case of endogeneity. 

In [ ]:
import scipy.stats as iid

In [ ]:
def dgp(N, true_beta, cov_ze=0):
    # define covariance mat of XEZ:
    cov_XEZZ = np.array(
        [[4, 1, 1, 2],
        [1, 2, cov_ze, cov_ze*0.5],
        [1, cov_ze, 3, 0.2],
        [2, cov_ze*.5, 0.2, 8]])*0.1
    
    XEZZ = iid.multivariate_normal(mean=[.2, 0, 0, 0],
                                   cov=cov_XEZZ).rvs(size = N)
    
    X = XEZZ[:, [0]]
    
    e = XEZZ[:, [1]]
    
    Z = XEZZ[:, 2:]
    
    y = ...
    X = np.c_[np.ones(shape = (N,1)), X] # add intercept
    Z = np.c_[np.ones(shape = (N,1)), Z]
    
    return (y, X, Z)

In [ ]:
true_beta = np.array([.2, .1])
N = 10000

# exogenous DGP
data_exog = dgp(N, true_beta)
# endogenous DGP
data_endog = dgp(N, true_beta, cov_ze = 1)

In [ ]:
# test IV estimate under exogeneity: true_beta = .1
from numpy.linalg import pinv

print(pinv(data_exog[2].T@(data_exog[1]))@
      (data_exog[2].T)@(data_exog[0]))

# Note this is close (except the intercept)!
# but the SEs are going to be all wrong!!

In [ ]:
# test IV estimate when Z is endogenous: true_beta = .1
print(pinv(data_endog[2].T@(data_endog[1]))@
      (data_endog[2].T)@(data_endog[0]))

In [ ]:
# let's look at our moment conditions
gj(true_beta, data_exog)

So far we have a DGP capable of making data for which our IV moment condition is satisfied and for which it is not. Let's see how GMM performs for each.

In [ ]:
# 1. instantiate
b0 = np.array([[0,0]]).T

gmm_endog = GMM(gj,data_endog,b0)
gmm_exog = GMM(gj,data_exog,b0)

# 2. estimate
print(f"true beta = {true_beta}")
print(f"beta estimate for endog. DGP: {gmm_endog.two_step_gmm()}")
print(f"beta estimate for exog. DGP: {gmm_exog.two_step_gmm()}")

In [ ]:
# Should we accept these models?
J_crit = limiting_J = iid.chi2(3-1).isf(0.05)

J_exog = ...
J_endog = ...


print(f"Critical stat for rejecting null of J: {J_crit:.4f}")
print(f"J_stat_endog = {J_endog:.4f}, J_stat_exog = {J_exog:.4f}")